# Removed CIs (BMC Discovery)

This notebook reproduces the DisMAL `removed` report using the raw CSV exports generated by the CLI.
It reads `raw_exports/<appliance>/recently_removed_cis.csv` for preview and optional re-export to the standard output folders.


## Requirements

We rely on `pandas` for tabular wrangling and `PyYAML` for configuration.
Uncomment the next cell to install them in your environment if needed.


In [ ]:
# %pip install -q pandas pyyaml

import re
from pathlib import Path

import pandas as pd
import yaml


## Configuration (from config.yaml)

Loads target details, resolves the output folder, and points to the matching raw export directory for each appliance.


In [ ]:
from pathlib import Path
import re
import yaml

def load_config_params(
    start: Path,
    appliance_name: str = None,
    appliance_index: int = 0,
) -> dict:
    def _find_repo_root(start: Path) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'config.yaml').exists():
                return p
        return start.parent

    def _slugify(value: str) -> str:
        return re.sub(r'[^A-Za-z0-9]+', '_', value).strip('_').lower() or 'default'

    repo_root = _find_repo_root(start)
    config_path = repo_root / 'config.yaml'

    with open(config_path, 'r') as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get('appliances') or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get('name') == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
    token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, 'r') as tf:
            token = tf.read().strip()
    if not token:
        token = None

    api_version = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
    verify_ssl = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f'output_{sanitized}'
    output_dir.mkdir(parents=True, exist_ok=True)

    export_name = ((selected or {}).get('name') or appliance_name or sanitized)
    raw_export_dir = repo_root / 'raw_exports' / _slugify(export_name)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
        "raw_export_dir": raw_export_dir,
    }


In [ ]:
def init_appliance(appliance_name: str = "prod"):
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)

    target = params["target"]
    api_version = params["api_version"]
    verify_ssl = params["verify_ssl"]
    output_dir = params["output_dir"]
    raw_export_dir = params["raw_export_dir"]

    print('Appliance Name :', appliance_name)
    print('Target         :', target)
    print('API Version    :', api_version)
    print('Verify SSL     :', verify_ssl)
    print('Raw CSV folder :', raw_export_dir)
    print('Output folder  :', output_dir)

    return {
        "params": params,
        "target": target,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
        "raw_export_dir": raw_export_dir,
        "appliance_name": appliance_name,
    }


# Initialise Instances


In [ ]:
print('Initialise Prod:')
twprod = init_appliance('prod')

print('Initialise Dev:')
twdev = init_appliance('dev')


In [ ]:
BASE_EXPORT_COLUMNS = ['Appliance Target', 'Appliance Name', 'Query Title']
EXPECTED_COLUMNS = [
    'kind',
    'name',
    'hashed_name',
    'os',
    'Last Successful IP',
    'Last Successful Scan',
    'Previous Found',
    'Next Found',
    'next kind',
    'prv kind',
    'Destroyed When',
]

def load_removed_cis(instance):
    csv_path = instance['raw_export_dir'] / 'recently_removed_cis.csv'
    if not csv_path.exists():
        raise FileNotFoundError(f"Missing export {csv_path}")

    df = pd.read_csv(csv_path)
    df = df.drop(columns=[c for c in BASE_EXPORT_COLUMNS if c in df.columns], errors='ignore')

    for col in EXPECTED_COLUMNS:
        if col not in df.columns:
            df[col] = pd.NA

    df.insert(0, 'Discovery Instance', instance['target'])
    ordered = ['Discovery Instance'] + EXPECTED_COLUMNS
    remainder = [c for c in df.columns if c not in ordered]
    return df[[c for c in ordered if c in df.columns] + remainder]


In [ ]:
prod_df = load_removed_cis(twprod)
print(twprod['target'])
display(prod_df.head(5))

dev_df = load_removed_cis(twdev)
print(twdev['target'])
display(dev_df.head(5))


In [ ]:
from pathlib import Path
import pandas as pd

def save(df: pd.DataFrame, output_dir: Path, filename: str):
    output_csv = output_dir / f"{filename}.csv"
    df.to_csv(output_csv, index=False)
    print(f"Saved to {output_csv}")


In [ ]:
report_name = 'removed_cis'

save(prod_df, twprod['output_dir'], report_name)
save(dev_df, twdev['output_dir'], report_name)
